In [ ]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import datetime

In [ ]:
train = os.path.join(path, 'train')
val = os.path.join(path, 'validation')
img_size = (150, 150)

train_data = tf.keras.utils.image_dataset_from_directory(train, shuffle = True, batch_size = train_size, image_size = img_size)
val_data = tf.keras.utils.image_dataset_from_directory(val, shuffle = True, batch_size = val_size, image_size = img_size)

val_batches = tf.data.experimental.cardinality(val_data)
test_data = val_data.take(val_batches // 5)
val_data = val_data.skip(val_batches // 5)

auto = tf.data.AUTOTUNE

train_data = train_data.prefetch(buffer_size=auto)
val_data = val_data.prefetch(buffer_size=auto)
test_data = test_data.prefetch(buffer_size=auto)

In [ ]:
base_model = tf.keras.application.ResNet50V2(
    input_shape = img_size, 
    include_top = False, 
    weight='imagenet'
)

feature_layer = hub.KerasLayer(
    base_model,
    input_shape=(150,150,3),
    trainable=False
)

model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.Dense(num_classes, activation = "softmax")
])
model.summary()
        
base_learning_rate = 0.0001
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)
model.summary()

In [ ]:
history = self.model.fit(
    self.train_data,
    epochs = self.e,
    validation_data=self.val_data
)